<a href="https://colab.research.google.com/github/arriyan-aa/AI-Hand-Computer-Vision/blob/main/cv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b>AI-Hand-Computer-Vision </b> </center> </h1>

In this tutorial, we will see how image datasets with different types of tasks

Distinguishing between AI-generated hands vs. real hands. Our project aims to address the issue of AI in art and other media. This is an important problem because the use of AI generated material in art and media is becoming increasingly more common and has brought up discussions of ethics and copyright. This model would help distinguish real human hands versus AI generated ones.